In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install --quiet scvi-colab
from scvi_colab import install
install()

/home/chels/anaconda3/envs/thesis/lib/python3.8/site-packages/scvi_colab/_core.py:41: UserWarning: 
                Not currently in Google Colab environment.

                Please run with `run_outside_colab=True` to override.

                Returning with no further action.
                
  warn(


In [3]:
import numpy as np
import pandas as pd
import scanpy as sc
import scvelo as scv
import torch
from velovi import preprocess_data, VELOVI
import scarches as sca

import matplotlib.pyplot as plt
import seaborn as sns

Global seed set to 0
/home/chels/anaconda3/envs/thesis/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/home/chels/anaconda3/envs/thesis/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


Get AnnData

In [4]:
adata = scv.datasets.pancreas()

In [5]:
scv.pp.filter_and_normalize(adata, min_shared_counts=30, n_top_genes=2000)
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)

Filtered out 21611 genes that are detected 30 counts (shared).
Normalized count data: X, spliced, unspliced.
Extracted 2000 highly variable genes.
Logarithmized X.
computing neighbors
    finished (0:00:29) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:02) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


In [6]:
adata = preprocess_data(adata)

computing velocities
    finished (0:00:02) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)


Get annotations

In [7]:
import gdown
url = 'https://drive.google.com/uc?id=1136LntaVr92G1MphGeMVcmpE0AqcqM6c'
output = 'reactome.gmt'
gdown.download(url, output, quiet=False)

KeyboardInterrupt: 

In [8]:
sca.utils.add_annotations(adata, 'reactome.gmt', min_genes=12, clean=True)

In [9]:
#Remove all genes not present in annotations

adata._inplace_subset_var(adata.varm['I'].sum(1)>0)

In [10]:
#Filter out terms with less than 12 genes 

select_terms = adata.varm['I'].sum(0)>12
adata.uns['terms'] = np.array(adata.uns['terms'])[select_terms].tolist()
adata.varm['I'] = adata.varm['I'][:, select_terms]

In [ ]:
#TODO: Use only highly variable genes

Instantiate model and train

In [17]:
VELOVI.setup_anndata(adata, spliced_layer="Ms", unspliced_layer="Mu")
vae = VELOVI(adata)

	with soft mask.
Last Decoder layer: softmax


In [18]:
vae.train()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Epoch 1/500:   0%|          | 0/500 [00:00<?, ?it/s]Init the group lasso proximal operator for the main terms.
local_loss: 1080.609619140625
loss: 1213.166259765625
local_loss: 1094.577880859375
loss: 1227.134521484375
local_loss: 1120.4163818359375
loss: 1252.9730224609375
local_loss: 1122.8966064453125
loss: 1255.4532470703125
local_loss: 1108.590087890625
loss: 1241.146728515625
local_loss: 1123.642822265625
loss: 1256.199462890625
local_loss: 1119.1746826171875
loss: 1251.7313232421875
local_loss: 1118.6484375
loss: 1251.205078125
local_loss: 1111.8660888671875
loss: 1244.4227294921875
local_loss: 1120.597900390625
loss: 1253.154541015625
local_loss: 1127.2506103515625
loss: 1259.8072509765625
local_loss: 1117.180908203125
loss: 1249.737548828125
local_loss: 1095.2568359375
loss: 1227.8134765625
Epoch 2/500:   0%|          | 1/500 [01:40<13:56:34, 100.59s/it, loss=nan, v_num=1]local_loss: 1125.078857421875
loss: 1257.3040771484375
local_loss: 1127.2822265625
loss: 1259.507446289062